In [11]:
using Plots
using NPZ
import Distributions; dst = Distributions
import DataStructures

struct Lattice
    memory::Dict{Tuple{Int16,Int16,Int16},Int16})
    neib::Tuple
end

function Lattice(lattice::Lattice, key::Tuple, value)
    lattice.memory[key] = value
end

function Lattice(lattice::Lattice, key::Tuple)
    if ~haskey(lattice.memory, key)
        lattice.memory[key] = 1
    end
    return lattice.memory[key]
end

function Lattice()
    return Lattice(Dict((0,0,0)=>0), (
        ( 1, 0, 0), 
        ( 0, 1, 0), 
        ( 0, 0, 1), 
        (-1, 0, 0), 
        ( 0,-1, 0), 
        ( 0, 0,-1)))
end

function len(lattice::Lattice)
    return length(lattice.memory)
end

function Neib(lattice::Lattice, position::Tuple)
    return [Lattice(lattice, position .+ neib_node) for neib_node in lattice.neib]
end

function Update!(lattice::Lattice, position::Tuple)
    for neib_node in lattice.neib
        if Lattice(lattice, position .+ neib_node) != 0
            temp = Neib(lattice, position .+ neib_node)
            if sum(temp .== 1) < length(temp)
                Lattice(lattice, position .+ neib_node, sum(temp .== 0)+1)
            end
        end
    end
    Lattice(lattice, position,0)
end

function Next(lattice, position)
    Update!(lattice, position)
end

   

function InitSim()
    lattice = Lattice()
    pos = (0,0,0)
    Lattice(lattice, pos, 0)
    for neib in lattice.neib
        Lattice(lattice, pos .+ neib, Lattice(lattice, pos .+ neib) + 1)
    end
    lattice_weigths = [Lattice(lattice, pos .+ neib) for neib in lattice.neib]
    weigths = Ω.(lattice_weigths; a=4, b=2)
    probs = weigths ./ sum(weigths)
    return lattice, probs, pos
end

function Update!(lattice, probs, pos, aa, bb)
    step = lattice.neib[rand(dst.Categorical(probs))]
    if Lattice(lattice, pos .+ step) == 0
        pos = pos .+ step
        lattice_weigths = [Lattice(lattice, pos .+ neib) for neib in lattice.neib]
        weigths = Ω.(lattice_weigths; a=aa, b=bb)
        probs = weigths ./ sum(weigths)
    else
        pos = pos .+ step
        Lattice(lattice, pos, 0)
        for neib in lattice.neib
            if Lattice(lattice, pos .+ neib) != 0
                Lattice(lattice, pos .+ neib, Lattice(lattice, pos .+ neib) + 1)
            end
        end
        lattice_weigths = [Lattice(lattice, pos .+ neib) for neib in lattice.neib]
        weigths = Ω.(lattice_weigths; a=aa, b=bb)
        probs = weigths ./ sum(weigths)    
    end
    return probs, pos
end


const aa = exp(3.5)
const bb = exp(2)
lattice, probs, pos = InitSim()
len_lattice = len(lattice)
count = 0

idx = 0
while len(lattice) < 100
    global probs
    global pos
    global len_lattice, idx
    probs, pos = Update!(lattice, probs, pos, aa, bb)
    if len(lattice) > len_lattice 
        len_lattice = len(lattice)
        idx += 1
        if idx % 100 == 0
            println(len_lattice)
        end
    end 
end

LoadError: syntax: extra token ")" after end of expression

In [20]:
2.0f0

2.0f0

In [18]:
one(Float32)

1.0f0

In [21]:
function Ω(x::Int16; a::Float32=1.0f0, b::Float32=1.0f0)
    """
    # Отображаем lattice_weigths в weigths используя параметры a и b
    0 => a
    2 => 1
    3:7 => b
    """
    if x == Int16(1)
        return a
    elseif x == Int16(2)
        return 1.0f0
    else
        return b
    end
end 

Ω (generic function with 1 method)

In [22]:
function InitSim()
    lattice = Lattice()
    for neib in lattice.neib
        Lattice(lattice, pos .+ neib, Lattice(lattice, pos .+ neib) + 1)
    end
    lattice_weigths = [Lattice(lattice, pos .+ neib) for neib in lattice.neib]
    weigths = Ω.(lattice_weigths; a=4, b=2)
    probs = weigths ./ sum(weigths)
    return lattice, probs, pos
end

InitSim (generic function with 1 method)

In [ ]:
const aa = exp(3.5)
const bb = exp(2)
lattice, probs, pos = InitSim()
len_lattice = len(lattice)
count = 0

idx = 0
while len(lattice) < 100
    global probs
    global pos
    global len_lattice, idx
    probs, pos = Update!(lattice, probs, pos, aa, bb)
    if len(lattice) > len_lattice 
        len_lattice = len(lattice)
        idx += 1
        if idx % 100 == 0
            println(len_lattice)
        end
    end 
end

In [23]:
@code_llvm InitSim()


;  @ In[22]:2 within `InitSim'
; Function Attrs: noreturn uwtable
define void @julia_InitSim_18002() #0 {
top:
  %0 = call nonnull %jl_value_t addrspace(10)* @jl_apply_generic(%jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 317903472 to %jl_value_t*) to %jl_value_t addrspace(10)*), %jl_value_t addrspace(10)** null, i32 0)
  call void @llvm.trap()
  unreachable
}


In [16]:
"""
;  @ In[12]:2 within `InitSim'
; Function Attrs: noreturn uwtable
define void @julia_InitSim_17856() #0 {
top:
  %0 = call nonnull %jl_value_t addrspace(10)* @jl_apply_generic(%jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 317903472 to %jl_value_t*) to %jl_value_t addrspace(10)*), %jl_value_t addrspace(10)** null, i32 0)
  call void @llvm.trap()
  unreachable
}
"""

";  @ In[12]:2 within `InitSim'\n; Function Attrs: noreturn uwtable\ndefine void @julia_InitSim_17856() #0 {\ntop:\n  %0 = call nonnull %jl_value_t addrspace(10)* @jl_apply_generic(%jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 317903472 to %jl_value_t*) to %jl_value_t addrspace(10)*), %jl_value_t addrspace(10)** null, i32 0)\n  call void @llvm.trap()\n  unreachable\n}\n"

In [4]:
function Lattice()
    return Lattice(Dict((0,0,0)=>0), (
        ( 1, 0, 0), 
        ( 0, 1, 0), 
        ( 0, 0, 1), 
        (-1, 0, 0), 
        ( 0,-1, 0), 
        ( 0, 0,-1)))
end

Lattice

In [5]:
npzwrite("res.npy", res)

In [1]:
struct Lattice
    memory::Dict{Tuple{Int16,Int16,Int16},Int16}
    neib::Tuple{Tuple{Int16,Int16,Int16},
    Tuple{Int16,Int16,Int16},
    Tuple{Int16,Int16,Int16},
    Tuple{Int16,Int16,Int16},
    Tuple{Int16,Int16,Int16},
    Tuple{Int16,Int16,Int16}}
end

In [8]:
const n0 = zero(Int16)
const n1 = one(Int16)
@code_lowered Lattice(
    Dict((n0,n0,n0)=>n0), 
    (
        ( n1, n0, n0), 
        ( n0, n1, n0), 
        ( n0, n0, n1), 
        ( -n1, n0, n0),  
        ( n0, -n1, n0),  
        ( n0, n0, -n1), 
    )
)

CodeInfo(
1 ─ %1 = %new(Main.Lattice, memory, neib)
└──      return %1
)

In [7]:
@code_llvm x::Int16 = 1;

ErrorException: expression is not a function call, or is too complex for @code_llvm to analyze; break it down to simpler parts if possible

In [10]:
zero(Int16)

Int16

In [12]:
typeof(-one(Int16))

Int16

In [6]:
@code_llvm Lattice()


;  @ In[4]:2 within `Lattice'
; Function Attrs: noreturn uwtable
define nonnull %jl_value_t addrspace(10)* @japi1_Lattice_17435(%jl_value_t addrspace(10)*, %jl_value_t addrspace(10)**, i32) #0 {
top:
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 3
  %3 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %3, i8 0, i32 24, i32 0, i1 false)
  %4 = alloca %jl_value_t addrspace(10)**, align 8
  store volatile %jl_value_t addrspace(10)** %1, %jl_value_t addrspace(10)*** %4, align 8
  %5 = call %jl_value_t*** inttoptr (i64 1720369504 to %jl_value_t*** ()*)() #6
; ┌ @ dict.jl:123 within `Dict' @ dict.jl:101
   %6 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
   %7 = bitcast %jl_value_t addrspace(10)** %6 to i64*
   store i64 2, i64* %7
   %8 = getelementptr %jl_value_t**, %jl_value_t*** %5, i32 0
   %9 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
   %10 = bitcast 

In [11]:
using Profile

In [14]:
Profile.@profile Update!(Lattice(), (1,0,0))

0

In [15]:
using BenchmarkTools

In [ ]:
aa = exp(3.5)
bb = exp(2)
lattice, probs, pos = InitSim()
len_lattice = len(lattice)
count = 0

idx = 0
len_lattice = len(lattice)
while len(lattice) < 1000000
    global probs
    global pos
    global len_lattice, idx
    probs, pos = Update!(lattice, probs, pos, aa, bb)
    if len(lattice) > len_lattice 
        len_lattice = len(lattice)
        idx += 1
        if idx % 100 == 0
            println(len_lattice)
        end
    end 
end

In [23]:
@benchmark 
    aa = exp(3.5)
    bb = exp(2)
    lattice, probs, pos = InitSim()
    len_lattice = len(lattice)
    count = 0

    idx = 0
    len_lattice = len(lattice)
    while len(lattice) < 100
        global probs
        global pos
        global len_lattice, idx
        probs, pos = Update!(lattice, probs, pos, aa, bb)
        if len(lattice) > len_lattice 
            len_lattice = len(lattice)
            idx += 1
            if idx % 100 == 0
                println(len_lattice)
            end
        end 
    end
end

LoadError: syntax: invalid "do" syntax